In [1]:
!pip install langdetect
!pip install googletrans==4.0.0-rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5331a9ecea220201dd5cb31f3198c7b75fe79cdd9bc82c118338d51128ad852d
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
import unicodedata
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from langdetect import detect
from googletrans import Translator

In [3]:
df = pd.read_excel("merged_chatbot_dataset.xlsx")
df.head()

,intent,response,language,source_file,source_sheet,example
0,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,What is the best appliance brand?
1,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,Which appliance brand should I buy?
2,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,Best company for home appliances?
3,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,What brand is good for electronics?
4,best_appliance_brand,"Top brands: El-Araby, B.TECH, Zanussi.",en,chatbot_dataset_english.xlsx,Sheet1,Recommend a good appliance brand.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   intent        400 non-null    object
 1   response      400 non-null    object
 2   language      400 non-null    object
 3   source_file   400 non-null    object
 4   source_sheet  400 non-null    object
 5   example       400 non-null    object
dtypes: object(6)
memory usage: 18.9+ KB


In [5]:
texts = df["example"].astype(str).tolist()
labels = df["intent"].tolist()

In [6]:
print(f"Number of texts: {len(texts)}")
print(f"Number of labels: {len(labels)}")

Number of texts: 400
Number of labels: 400


In [7]:
print(texts)

['What is the best appliance brand?', 'Which appliance brand should I buy?', 'Best company for home appliances?', 'What brand is good for electronics?', 'Recommend a good appliance brand.', 'Where can I buy ceramics?', 'Best place to buy tiles?', 'Top ceramic stores?', 'Where to find good ceramic options?', 'Recommend a ceramic shop.', 'Where can I find cheap appliances?', 'Best place for low‑price appliances?', 'Where to buy affordable electronics?', 'Any discounts on appliances?', 'Cheapest store for home devices?', 'What are the best TV brands?', 'Which TV brand should I buy?', 'Recommend a good TV brand.', 'Best TV for quality?', 'Top recommended TVs?', 'What is the best fridge brand?', 'Which fridge should I buy?', 'Top refrigerator brands?', 'Recommend a good fridge.', 'Best long‑lasting fridge?', 'What is the best AC brand?', 'Which AC should I buy?', 'Top air conditioner brands?', 'Best AC for long use?', 'Recommend a good AC.', 'What is the best washing machine?', 'Which washi

In [8]:
print(labels)

['best_appliance_brand', 'best_appliance_brand', 'best_appliance_brand', 'best_appliance_brand', 'best_appliance_brand', 'best_place_buy_ceramics', 'best_place_buy_ceramics', 'best_place_buy_ceramics', 'best_place_buy_ceramics', 'best_place_buy_ceramics', 'cheapest_appliance_store', 'cheapest_appliance_store', 'cheapest_appliance_store', 'cheapest_appliance_store', 'cheapest_appliance_store', 'best_tv_brands', 'best_tv_brands', 'best_tv_brands', 'best_tv_brands', 'best_tv_brands', 'best_fridge_brands', 'best_fridge_brands', 'best_fridge_brands', 'best_fridge_brands', 'best_fridge_brands', 'best_ac_brands', 'best_ac_brands', 'best_ac_brands', 'best_ac_brands', 'best_ac_brands', 'best_washer_brands', 'best_washer_brands', 'best_washer_brands', 'best_washer_brands', 'best_washer_brands', 'best_kitchen_stores', 'best_kitchen_stores', 'best_kitchen_stores', 'best_kitchen_stores', 'best_kitchen_stores', 'home_tools_store', 'home_tools_store', 'home_tools_store', 'home_tools_store', 'home_too

In [9]:
def normalize_arabic(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
texts = [normalize_arabic(text) if any('\u0600' <= char <= '\u06FF' for char in text) else text for text in texts]

In [10]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [11]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post')

vocab_size = len(tokenizer.word_index) + 1
num_classes = len(set(labels))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, y, test_size=0.2, random_state=42
)

In [13]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [21]:
vocab_size = 10000
embedding_dim = 128
gru_units = 128
num_classes = len(label_encoder.classes_)
max_len = 50


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),

    tf.keras.layers.GRU(gru_units, return_sequences=False),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)



In [22]:
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 50, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,387,978 (5.29 MB)

 Trainable params: 1,387,978 (5.29 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.1204 - loss: 2.3037 - val_accuracy: 0.0625 - val_loss: 2.3113
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.1252 - loss: 2.3058 - val_accuracy: 0.0625 - val_loss: 2.3152
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.1243 - loss: 2.3105 - val_accuracy: 0.0625 - val_loss: 2.3170
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.1308 - loss: 2.3078 - val_accuracy: 0.0625 - val_loss: 2.3165
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.0994 - loss: 2.3100 - val_accuracy: 0.0625 - val_loss: 2.3153
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.0806 - loss: 2.3022 - val_accuracy: 0.0625 - val_loss: 2.3164
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.1057 - loss: 2.2955 - val_accuracy: 0.0625 - val_loss: 2.3161
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.1254 - loss: 2.3254 - val_accuracy: 0.0312 - val_loss: 2.314

In [32]:
y_pred = np.argmax(model.predict(X_test), axis=1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [33]:
print(y_pred)

[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5]


In [34]:
print(accuracy_score(y_test, y_pred)*100)

6.25


In [35]:
print(mean_absolute_error(y_test, y_pred))

2.6375


In [41]:
print(mean_absolute_percentage_error(y_test, y_pred))

3096224743817216.0


In [42]:
print(mean_squared_error(y_test, y_pred))

9.2625


In [36]:
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

                          precision    recall  f1-score   support

          best_ac_brands       0.00      0.00      0.00        11
    best_appliance_brand       0.00      0.00      0.00         5
      best_fridge_brands       0.00      0.00      0.00        12
     best_kitchen_stores       0.00      0.00      0.00         5
 best_place_buy_ceramics       0.00      0.00      0.00         8
          best_tv_brands       0.06      1.00      0.12         5
      best_washer_brands       0.00      0.00      0.00        12
          ceramic_prices       0.00      0.00      0.00         6
cheapest_appliance_store       0.00      0.00      0.00         6
        home_tools_store       0.00      0.00      0.00        10

                accuracy                           0.06        80
               macro avg       0.01      0.10      0.01        80
            weighted avg       0.00      0.06      0.01        80



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [37]:
def predict_intent(text):
    text = normalize_arabic(text) if any('\u0600' <= char <= '\u06FF' for char in text) else text
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=50, padding='post')
    pred = model.predict(pad)
    intent = label_encoder.inverse_transform([np.argmax(pred)])
    return intent[0]

In [38]:
def get_response(text):
    print(f"Input text: {text}")

    # Detect language
    try:
        input_lang = detect(text)
        print(f"Detected language: {input_lang}")
    except:
        input_lang = 'en'
        print("Language detection failed, defaulting to 'en'")

    predicted_intent = predict_intent(text)
    print(f"Predicted intent: {predicted_intent}")

    # Filter responses for the predicted intent and matching language
    matching_responses = df[(df['intent'] == predicted_intent) & (df['language'] == input_lang)]
    print(f"Matching responses for intent '{predicted_intent}' and language '{input_lang}': {len(matching_responses)} found")

    if not matching_responses.empty:
        response_text = matching_responses['response'].iloc[0]
        print(f"Selected response: {response_text}")
    else:
        print("No matching response found. Checking for English fallback...")
        english_responses = df[(df['intent'] == predicted_intent) & (df['language'] == 'en')]
        if not english_responses.empty:
            response_text = english_responses['response'].iloc[0]
            print(f"English fallback response: {response_text}")
        else:
            print("No English response either. Using any available or translating...")
            any_response = df[df['intent'] == predicted_intent]['response'].iloc[0] if not df[df['intent'] == predicted_intent].empty else "Sorry, I don't have a response for that intent."
            if input_lang == 'ar':
                translator = Translator()
                try:
                    response_text = translator.translate(any_response, src='en', dest='ar').text
                    print(f"Translated response: {response_text}")
                except:
                    response_text = any_response
                    print("Translation failed, using original.")
            else:
                response_text = any_response

    return response_text


In [39]:
print(get_response("ايه افضل شركة اجهزة كهربائية؟"))

Input text: ايه افضل شركة اجهزة كهربائية؟
Detected language: ar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted intent: best_tv_brands
Matching responses for intent 'best_tv_brands' and language 'ar': 20 found
Selected response: أفضل الماركات: سامسونج – LG – توشيبا
أفضل الماركات: سامسونج – LG – توشيبا


In [40]:
print(get_response("Where can I buy ceramics?"))

Input text: Where can I buy ceramics?
Detected language: en
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicted intent: best_tv_brands
Matching responses for intent 'best_tv_brands' and language 'en': 20 found
Selected response: Top TV brands: Samsung, LG, Toshiba.
Top TV brands: Samsung, LG, Toshiba.
